<a href="https://colab.research.google.com/github/ludocomito/marrtino-face-detection/blob/main/MARRtino_Face_Detection_Use_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Face mask classification model testing
The purpose of this notebook is to give an interactive environment to test the face mask classifier (find the repo [here](https://github.com/ludocomito/marrtino-face-detection)). In particular you can choose to upload any custom image, or to directly test a photo taken from your webcam.

In [15]:
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, models, transforms
import os

Import the trained model (should be in a .pth extension):

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 3)
model = model.to(device)
model.load_state_dict(torch.load('mask_recognition_model.pth', map_location=device))

In [ ]:
# Sets the model to evaluation mode
model.eval()

The following is a code snippet provided by Google Colab. It estblishes a connection with your webcam feed and shows you an interactive interface that lets you take a photo directly on the notebook.

In [84]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))
  
  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

Here we load and process the image we want to classify, then we use the model to make the prediction and print it.

In [89]:
from PIL import Image
from torch.autograd import Variable

loader = transforms.Compose([transforms.ToTensor(),
                             transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

def image_loader(image_name):
    """load image, returns cuda tensor"""
    image = Image.open(image_name).convert('RGB')
    image = loader(image).float()
    image = Variable(image, requires_grad=True)
    image = image.unsqueeze(0)  #this is for VGG, may not be needed for ResNet
    return image  #assumes that you're using GPU

In [90]:
image = image_loader('photo.jpg')
classes = ['mask_weared_incorrect', 'with_mask', 'without_mask']
pred = model(image)
print(f'The output straight from the prediction is: \n {pred}')

_, predicted = torch.max(pred, 1)
print(f'The model predicted: {classes[predicted]}')


The output straight from the prediction is: 
 tensor([[-9.6810,  5.0670,  3.1634]], grad_fn=<AddmmBackward0>)
The model predicted: with_mask
